In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv
import json
import numpy as np
from random import randint
import pandas as pd
import googlemaps
from geotext import GeoText

In [2]:
gmapKey =
gmaps = googlemaps.Client(key=gmapKey)

In [4]:
provfile = '../data/BeninArtifacts.csv'
df = pd.read_csv(provfile)
df['NextProvNum'] = df['ProvNum'].shift(-1)

In [44]:
all_properties = []
all_geometry = []
curr_item_properties = []
curr_item_geometry = []
curr_item_provenance = []
for ii, row in df.iterrows():
    if row['ProvNum'] == 0:
        prop_line = []
        prop_line.append('"index": "'+str(len(all_properties))+'"')
        prop_line.append('"item": "'+str(row['Item'])+'"')
        prop_line.append('"name": "'+str(row['Name'])+'"')
        prop_line.append('"institution": "'+str(row['Location'])+'"')
        prop_line.append('"img_url": "'+str(row['IMG'])+'"')
        prop_line.append('"item_link": "'+str(row['Link'])+'"')
        all_properties.append(', '.join(prop_line))
    curr_prop = []
    curr_prop.append('"index": "'+str(row['ProvNum'])+'"')
    curr_prop.append('"item": "'+str(row['Item'])+'"')
    curr_prop.append('"provenance": "'+str(row['Provenance'])+'"')
    curr_item_provenance.append('"provenance": "'+str(row['Provenance'])+'"')
    curr_prop.append('"place": "'+str(row['place'])+'"')
    curr_prop.append('"yearStr": "'+str(row['year'])+'"')
    curr_prop.append('"year": "'+str(row['year']).split('/')[-1]+'"')
    curr_item_properties.append(', '.join(curr_prop))
    geocode_result = gmaps.geocode(row['place'])
    ECo,NCo = geocode_result[0]['geometry']['location'].values()
    curr_item_geometry.append(str(NCo+randint(-10,10)/200)+', '+str(ECo+randint(-10,10)/200))
    if row['NextProvNum'] == 0 or row['NextProvNum'] == np.nan:
        lines = []
        for ii,(prop,geom) in enumerate(zip(curr_item_properties,curr_item_geometry)):
            lines.append('{"type": "Feature", "id": '+str(ii)+', "properties": {'+prop+'}, "geometry": { "type": "Point","coordinates": ['+geom+'] } }')
        f = open('../data/'+str(row['Item'])+'.geojson', "w")
        f.write('{\n')
        f.write('"type": "FeatureCollection", \n')
        f.write('"features": [\n')
        f.write(',\n'.join(lines))
        f.write('\n]}')
        f.close()
        all_geometry.append(curr_item_geometry[-1])
        curr_item_properties = []
        curr_item_geometry = []
        curr_item_provenance = []

In [45]:
lines = []
for ii,(prop,geom) in enumerate(zip(all_properties,all_geometry)):
    lines.append('{"type": "Feature", "id": '+str(ii)+', "properties": {'+prop+'}, "geometry": { "type": "Point","coordinates": ['+geom+'] } }')
f = open('../data/all.geojson', "w")
f.write('{\n')
f.write('"type": "FeatureCollection", \n')
f.write('"features": [\n')
f.write(',\n'.join(lines))
f.write('\n]}')
f.close()